<img src="monai.png" style="width: 700px;"/>

Welcome to the MONAI bootcamp! This tutorial shows how to create an organ segmentation application for a PyTorch model that has been trained with MONAI.

### Using Google Colab

This notebook has the pip command for installing MONAI and will be added to any subsequent notebook.

**Required Packages for Colab Execution**

Execute the following cell to install MONAI the first time a colab notebook is run:

In [ ]:
!python -c "import clara" || pip install -qU "clara-viz"
!python -c "import itk" || pip install -qU "itk"
!python -c "import pydicom" || pip install -qU "pydicom"
!python -c "import monai" || pip install -qU "monai[nibabel]"
!python -c "import monai.deploy" || pip install -qU "monai-deploy-app-sdk"

**Note:** If you're having issues with imports, try to "Restart Kernel" for Jupyter from the "Kernel" dropdown menu.

**Enabling GPU Support**

To use GPU resources through Colab, change the runtime to GPU:

1. From the **"Runtime"** menu select **"Change Runtime Type"**
2. Choose **"GPU"** from the drop-down menu
3. Click **"SAVE"**

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not). Running

**!nvidia-smi**

in a cell will verify this has worked and show you what kind of hardware you have access to.    

In [ ]:
!nvidia-smi

# Creating a Segmentation App with MONAI Deploy App SDK

Deploying AI models requires integration with a clinical imaging network, even in a for-research-use setting. This requirement means that the AI deployment application will need to support standards-based imaging protocols like the DICOM protocol for Radiological imaging.

Typically, DICOM network communication, either in DICOM TCP/IP network protocol or DICOMWeb, would be handled by DICOM devices or services, e.g., MONAI Deploy Informatics Gateway. So the deployment application itself would only need to use DICOM Part 10 files as input and save the AI result in DICOM Part10 file(s). For segmentation, the DICOM instance file could be a DICOM Segmentation object or a DICOM RT Structure Set, and for classification, DICOM Structure Report and/or DICOM Encapsulated PDF.

During model training, input and label images are typically in non-DICOM volumetric image format, e.g., NIfTI and PNG, converted from a specific DICOM study series. Furthermore, the voxel spacings were most likely re-sampled to be uniform for all images. When integrated with imaging networks and receiving DICOM instances from modalities and Picture Archiving and Communications System, PACS, an AI deployment application may have to deal with a whole DICOM study with multiple series, whose images' spacing may not be the same as expected by the trained model. To address these cases consistently and efficiently, MONAI Deploy Application SDK provides classes, called operators, to parse DICOM studies, select specific series with application-defined rules, and convert the selected DICOM series into domain-specific image format along with meta-data representing the pertinent DICOM attributes.

The following sections will demonstrate how to create a MONAI Deploy application package using the MONAI Deploy App SDK.

---
 > **Note**: For local testing, if there is a lack of DICOM Part 10 files, one can use open source programs, e.g., 3D Slicer, to convert NIfTI to DICOM files.
---

### Contents
* [Setup](#setup)
* [Creating Operator Class](#create_op)
* [Executing App Locally](#execute_app_local)
* [Write Files to Disk](#write_files)
* [Run App using CLI](#run_cli)
* [Packing App as Docker Image](#package_app)
* [Clara Viz](#claraviz)
* [Exercise](#exercise)
* [Conclusion](#conclusion)

## Creating Operators and connecting them in the Application class

We will implement an application that consists of five Operators:

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: a folder path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
    - **Input(selection_rules)**: a selection rule (Dict)
    - **Output(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
- **DICOMSeriesToVolumeOperator**:
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **SpleenSegOperator**:
    - **Input(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output(seg_image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **DICOMSegmentationWriterOperator**:
    - **Input(seg_image)**: a segmentation image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(dicom_seg_instance)**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))


---
 > **Note**: The `DICOMSegmentationWriterOperator` needs both the segmentation image and the original DICOM series meta-data to use the patient demographics and the DICOM Study level attributes.
---

The workflow of the application would look like this.

![DICOM Operator Workflow](images/dicom_workflow.png)
```


<a id='setup'></a>
## 0 Setup

### 0.1 Setup environment

We'll set up a variable to point to the 03_files folder, where we're storing data throughout this notebook.

In [ ]:
NOTEBOOK_ROOT="data/03_files/"

**Note:** If you're having issues with imports, try to "Restart Kernel" for Jupyter from the "Kernel" dropdown menu.

#### Download Data required for all of the MONAI Deploy Notebooks
We're going to download sample images and python files required for running the MONAI Deploy Jupyter Notebooks.

In [ ]:
!wget https://github.com/zephyrie/monai-bootcamp/releases/download/monai-deploy-data-v0.1/deploy_data.zip
!unzip deploy_data.zip

### 0.2 Setup imports

Let's import necessary classes/decorators to define Application and Operator.

In [ ]:
import logging
from os import path

from numpy import uint8

import monai.deploy.core as md
from monai.deploy.core import ExecutionContext, Image, InputContext, IOType, Operator, OutputContext
from monai.deploy.operators.monai_seg_inference_operator import InMemImageReader, MonaiSegInferenceOperator
from monai.transforms import (
    Activationsd,
    AsDiscreted,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    Invertd,
    LoadImaged,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
    KeepLargestConnectedComponentd,
)

from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
# from monai.deploy.operators.clara_viz_operator import ClaraVizOperator

import clara.viz
from monai.deploy.operators.dicom_text_sr_writer_operator import DICOMTextSRWriterOperator, EquipmentInfo, ModelInfo
from typing import Text
import pydicom
import numpy as np

<a id='create_op'></a>
## 1. Creating Operator classes

### 1.1 Creating Model Specific Inference Operator classes

Each Operator class inherits [Operator](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html) class and input/output properties are specified by using [@input](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.input.html)/[@output](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.output.html) decorators.

Business logic would be implemented in the <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html#monai.deploy.core.Operator.compute">compute()</a> method.

The App SDK provides a `MonaiSegInferenceOperator` class to perform segmentation prediction with a Torch Script model. For consistency, this class uses MONAI dictionary-based transforms, as `Compose` object, for pre and post transforms. The model-specific inference operator will then only need to create the pre and post transform `Compose` based on what has been used in the model training and validation. Note that for deploy application, `ignite` is not needed nor supported.



### 1.2 SpleenSegOperator

The `SpleenSegOperator` gets as input an in-memory [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) object that has been converted from a DICOM CT series by the preceding `DICOMSeriesToVolumeOperator`, and as output in-memory segmentation [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) object.

The `pre_process` function creates the pre-transforms `Compose` object. For `LoadImage`, a specialized `InMemImageReader`, derived from MONAI `ImageReader`, is used to convert the in-memory pixel data and return the `numpy` array as well as the meta-data. Also, the DICOM input pixel spacings are often not the same as expected by the model, so the `Spacingd` transform must be used to re-sample the image with the expected spacing.

The `post_process` function creates the post-transform `Compose` object. The `SaveImageD` transform class is used to save the segmentation mask as NIfTI image file, which is optional as the in-memory mask image will be passed down to the DICOM Segmentation writer for creating a DICOM Segmentation instance. The `Invertd` must also be used to revert the segmentation image's orientation and spacing to be the same as the input.

When the `MonaiSegInferenceOperator` object is created, the `ROI` size is specified, as well as the transform `Compose` objects. Furthermore, the dataset image key names are set accordingly.

Loading of the model and performing the prediction are encapsulated in the `MonaiSegInferenceOperator` and other SDK classes. Once the inference is completed, the segmentation [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) object is created and set to the output (<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.OutputContext.html#monai.deploy.core.OutputContext.set">op_output.set(value, label)</a>), by the `MonaiSegInferenceOperator`.

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("seg_image", Image, IOType.IN_MEMORY)
@md.output("result_text", Text, IOType.IN_MEMORY)
@md.env(pip_packages=["monai==0.8.0", "torch>=1.5", "numpy>=1.20", "nibabel"])
class SpleenSegOperator(Operator):
    """Performs Spleen segmentation with a 3D image converted from a DICOM CT series.
    """

    def __init__(self):

        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()
        self._input_dataset_key = "image"
        self._pred_dataset_key = "pred"

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):

        input_image = op_input.get("image")
        if not input_image:
            raise ValueError("Input image is not found.")

        output_path = context.output.get().path

        # This operator gets an in-memory Image object, so a specialized ImageReader is needed.
        _reader = InMemImageReader(input_image)
        pre_transforms = self.pre_process(_reader)
        post_transforms = self.post_process(pre_transforms, path.join(output_path, "prediction_output"))

        # Delegates inference and saving output to the built-in operator.
        infer_operator = MonaiSegInferenceOperator(
            (160,160,160,),
            pre_transforms,
            post_transforms,
        )

        # Setting the keys used in the dictionary based transforms may change.
        infer_operator.input_dataset_key = self._input_dataset_key
        infer_operator.pred_dataset_key = self._pred_dataset_key

        # Now let the built-in operator handles the work with the I/O spec and execution context.
        infer_operator.compute(op_input, op_output, context)
        
    def pre_process(self, img_reader) -> Compose:
        """Composes transforms for preprocessing input before predicting on a model."""

        my_key = self._input_dataset_key
        return Compose(
            [
                LoadImaged(keys=my_key, reader=img_reader),
                EnsureChannelFirstd(keys=my_key),
                Spacingd(keys=my_key, pixdim=[1.0, 1.0, 1.0], mode=["bilinear"], align_corners=True),
                ScaleIntensityRanged(keys=my_key, a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
                CropForegroundd(keys=my_key, source_key=my_key),
                ToTensord(keys=my_key),
            ]
        )

    def post_process(self, pre_transforms: Compose, out_dir: str = "./prediction_output") -> Compose:
        """Composes transforms for postprocessing the prediction results."""

        pred_key = self._pred_dataset_key
        return Compose(
            [
                Activationsd(keys=pred_key, softmax=True),
                AsDiscreted(keys=pred_key, argmax=True),
                Invertd(keys=pred_key, transform=pre_transforms, orig_keys=self._input_dataset_key, nearest_interp=True),
                ###### uncomment line below for Exercise 3
                # KeepLargestConnectedComponentd(keys=pred_key,applied_labels=[1]),
                SaveImaged(keys=pred_key, output_dir=out_dir, output_postfix="seg", output_dtype=uint8, resample=False),
            ]
        )


### 1.3 Creating Application class

Our application class would look like below.

It defines `App` class, inheriting [Application](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html) class.

The requirements (resource and package dependency) for the App can be specified by using [@resource](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.resource.html) and [@env](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.env.html) decorators.

The base class method, `compose`, is overridden. Objects required for DICOM parsing, series selection (selecting the first series for the current release), pixel data conversion to volume image, and segmentation instance creation are created, so is the model-specific `SpleenSegOperator`. The execution pipeline, as a Directed Acyclic Graph, is created by connecting these objects through <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>.

We have multiple powerful operators in this application:
1. DICOMDataLoaderOperator
2. DICOMSeriesSelectorOperator
3. DICOMSeriesToVolumeOperator
4. SpleenSegOperator
5. DICOMSegmentationWriterOperator


In [ ]:
@resource(cpu=1, gpu=1, memory="7Gi")
class AISpleenSegApp(Application):  
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compose(self):

        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator()
        ### Line below is for the exercise 
        # series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # Creates DICOM Seg writer with segment label name in a string list
        dicom_seg_writer = DICOMSegmentationWriterOperator(seg_labels=["Spleen"])

        # Creates the model specific segmentation operator
        spleen_seg_op = SpleenSegOperator()

        # Creates the DAG by linking the operators
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"})
        self.add_flow(series_to_vol_op, spleen_seg_op, {"image": "image"})

        self.add_flow(series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"})
        self.add_flow(spleen_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})
        
        ####  For Exercise #3 uncomment lines below
        # volume_calculate_operator=CalculateVolumeOperator()
        # self.add_flow(spleen_seg_op,volume_calculate_operator, {"seg_image": "seg_image"})
        
        ####  For Exercise #4 add DICOM SR
        # my_model_info = ModelInfo("MONAI WG Trainer", "Segmentation models", "0.1", "xyz")
        # my_equipment = EquipmentInfo(manufacturer="MOANI Deploy App SDK", manufacturer_model="DICOM SR Writer")
        # my_special_tags = {"SeriesDescription": "Not for clinical use. The result is for research use only."}
        # dicom_sr_operator = DICOMTextSRWriterOperator(
        #     copy_tags=False, model_info=my_model_info, equipment_info=my_equipment, custom_tags=my_special_tags)        
        # self.add_flow(series_selector_op, dicom_sr_operator, {"study_selected_series_list": "study_selected_series_list"})
        # self.add_flow(volume_calculate_operator, dicom_sr_operator, {"result_text": "classification_result"})
        

<a id='execute_app_local'></a>
## 2. Executing app locally

We can execute the app in the Jupyter notebook. Note that the DICOM files of the CT Abdomen series must be present in the `dcm` and the Torch Script model at `model.ts`.

In [ ]:
app = AISpleenSegApp()

app.run(input=NOTEBOOK_ROOT+"/input", output=NOTEBOOK_ROOT+"/output", model=NOTEBOOK_ROOT+"/model.ts")

### 2.1 Check Output
Let's check out the output DCM files we produced. You can see we have a DICOM SEG, and after our exercise later in the notebook, we'll also have a DICOM SR.

In [ ]:
!ls -la {NOTEBOOK_ROOT}/output

**Note**: Below cell only used after exercise 7.4

In [ ]:
#uncomment below lines and replace dcmSRPath with file produced 
#dcmSRPath="1.2.826.0.1.3680043.8.498.51905453107544020300500494684680939405_SR.dcm"
#pydicom.read_file(NOTEBOOK_ROOT+"/output/"+dcmSRPath) 

<a id='write_file'></a>
## 3. Write code to Disk

Once the application is verified inside the Jupyter notebook, we can write the Python code into Python files in an application folder.

The application folder structure would look like below:

```bash
my_app
├── __main__.py
├── app.py
└── spleen_seg_operator.py
```

---
 > **Note**: We can create a single application Python file (such as `spleen_app.py`) that includes the files' content instead of creating multiple files.
You will see such an example in <a href="./02_mednist_app.html#executing-app-locally">MedNist Classifier Tutorial</a>.
---

We have already written the code in the cells above to a `my_app` directory.

The below lines are needed to execute the application code by using `python` interpreter.
```python
if __name__ == "__main__":
    AISpleenSegApp(do_run=True)
```

In [ ]:
!ls -la {NOTEBOOK_ROOT}/my_app

In [ ]:
!cat {NOTEBOOK_ROOT}/my_app/__main__.py

<a id='run_cli'></a>
## 4. Run App on CLI
You can run the App in one of two ways:
- Using the python command
- Using the 'monai-deploy' command 

---
 > **Note**: We are executing python code which makes developing and debugging simple.
---

In [ ]:
!python $NOTEBOOK_ROOT"/my_app" -i $NOTEBOOK_ROOT"/input" -o $NOTEBOOK_ROOT"/output" -m $NOTEBOOK_ROOT"/model.ts"

The above command and below command both run the application, but one uses python directly, and the other uses the provided MONAI Deploy CLI.

In [ ]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
!monai-deploy exec $NOTEBOOK_ROOT"/my_app" -i $NOTEBOOK_ROOT"/input" -o $NOTEBOOK_ROOT"/output" -m $NOTEBOOK_ROOT"/model.ts"

We'll then use the `ls -la` comand to list the output files.

In [ ]:
!ls -la  $NOTEBOOK_ROOT/output

<a id='package_app'></a>
## 5. Package app (creating MAP Docker image)

Let's package the app with [MONAI Application Packager](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/packaging_app.html).

Below we'll use the monai-deploy CLI to package the application by using the package command.  We'll pass the `my_app` folder path and use the `my_app` tag. We'll use the `-l DEBUG` option to see the progress.

In [ ]:
#!monai-deploy package -b nvcr.io/nvidia/pytorch:21.11-py3 $NOTEBOOK_ROOT"/my_app" --tag my_app:latest -m $NOTEBOOK_ROOT"/model.ts" -l DEBUG

After creating the docker image, we can verify it by using the `docker image ls` command and grepping for `my_app`.

In [ ]:
#!docker image ls | grep my_app

### 5.1 Executing packaged app locally

The packaged app can be run locally through [MONAI Application Runner](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/executing_packaged_app_locally.html).

In [ ]:
# Launch the app
#!monai-deploy run my_app:latest "${NOTEBOOK_ROOT}/input" "${NOTEBOOK_ROOT}/output"

### 5.2 Check the output from the packaged app
Now we can see the results from the packaged docker app.

In [ ]:
#!ls -la $NOTEBOOK_ROOT/output

<a id='claraviz'></a>
## 6. Visualize the Spleen Segmentation using Clara Viz

#### 6.1 Convert DICOM to NIfTI

First, we'll need to convert our DICOM results to NifTI. To do that, we'll use dcm2niix, a tool designed to convert neuroimaging data from the DICOM format to the NIfTI format. 

In [ ]:
!wget -q https://github.com/rordenlab/dcm2niix/releases/latest/download/dcm2niix_lnx.zip -O dcm2niix_lnx.zip
!unzip -o dcm2niix_lnx.zip && rm dcm2niix_lnx.zip

In [ ]:
!./dcm2niix -f %i -z y -o $NOTEBOOK_ROOT"output" $NOTEBOOK_ROOT"/input/"

#### 6.2 Visualize segmentation

[NVIDIA Clara Viz](https://github.com/NVIDIA/clara-viz) is a platform for visualizing 2D/3D medical imaging data. It enables building applications that leverage powerful volumetric visualization using CUDA-based ray tracing.

Clara Viz offers a Python Wrapper for rapid experimentation. It also includes a collection of visual widgets for performing interactive medical image visualization in Jupyter Lab notebooks which we'll use below.

In [ ]:
from clara.viz.core import DataDefinition
from clara.viz.widgets import Widget
from ipywidgets import interactive, Dropdown, Box, VBox

In [ ]:
imagePath=NOTEBOOK_ROOT+'/output/123456.nii.gz'
labelPath=NOTEBOOK_ROOT+'/output/prediction_output/1.2.826.0.1.3680043.2.1125.1/1.2.826.0.1.3680043.2.1125.1_seg.nii.gz'

data_definition = DataDefinition()
data_definition.append(imagePath,'DXYZ')
data_definition.append(labelPath, 'MXYZ')

In [ ]:
widget = Widget()
widget.select_data_definition(data_definition)
# default view mode is 'CINEMATIC' switch to 'SLICE_SEGMENTATION' since we have no transfer functions defined
widget.settings["Views"][0]["mode"] = "SLICE_SEGMENTATION"
widget.settings["Views"][0]["cameraName"] = "Top"
widget.set_settings()

# add controls
def set_view_mode(view_mode):
    widget.settings["Views"][0]["mode"] = view_mode
    if view_mode == "CINEMATIC":
        widget.settings["Views"][0]["cameraName"] = "Perspective"
    elif widget.settings["Views"][0]["cameraName"] == "Perspective":
        widget.settings["Views"][0]["cameraName"] = "Top"
    widget.set_settings()

widget_view_mode = interactive(
    set_view_mode,
    view_mode=Dropdown(
        options=[("Cinematic", "CINEMATIC"), ("Slice", "SLICE"), ("Slice Segmentation", "SLICE_SEGMENTATION")],
        value="SLICE_SEGMENTATION",
        description="View mode",
    ),
)

def set_camera(camera):
    if widget.settings["Views"][0]["mode"] != "CINEMATIC":
        widget.settings["Views"][0]["cameraName"] = camera
        widget.set_settings()
widget_set_camera = interactive(set_camera, camera=Dropdown(options=["Top", "Right", "Front"], value="Top", description="Camera"))

#### Clara Viz Controls

After running the cell below, you should see the visualization of the image and segmentation. 
- You can use the scroll wheel to zoom in/out
- Left-click hold then up down to go through the slices 
- Middle-click hold to pan

You can select a view from the drop-down menu to view:
- Render view
- Image only 
- Image and segmentation

In [ ]:
display(Box([widget, VBox([widget_view_mode, widget_set_camera])]))

<a id='exercise'></a>
## 7. Exercise

### 7.1 Clean up the segmentation using Connected Components

If you go back to the segmentation visualization and carefully examine it, you will see some small parts that aren't relevant and that we could remove. For this, you can use the KeepLargestConnectedComponent transform from MONAI Core. Check out [KeepLargestConnectedComponentd](https://docs.monai.io/en/stable/transforms.html).

**Hint**: you can add the line `KeepLargestConnectedComponentd(keys=pred_key,applied_labels=[1])` in the post-transformations of your app.

After adding the post-transform, re-run your app and confirm with Clara Viz that the small areas are removed.

### 7.2 Add DICOM Selection Rules
We mentioned earlier that you could pass rules to filter on the series your AI workflow should process. Now, go back and pass in a rule and re-run your code. 

**Hints**:
- In `AISpleenSegApp` try passing in `Sample_Rules_Text` to the `DICOMSeriesSelectorOperator` 
- Try changing the rule to MR and see what happens 

Below, you'll find a rules sample, written in JSON, for selecting a CT series. If the study has more than 1 CT series, then all of them will be selected. Please see more detail in [DICOMSeriesSelectorOperator](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.operators.DICOMSeriesSelectorOperator.html).
```
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""
```

In the `AISpleenSegApp` code above, you'll find comments for including the rule definition in the DICOM Series Selector. Make sure that you comment out the original DICOM Series Selector that doesn't have a parameter. You will also need to ensure that you instantiate a rule selection in a variable named `Sample_Rules_Text`.

```
series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
```

Now re-run your app and see how the series selector works.
You should also change the rules a bit and see how that would work.

### 7.3 Calculate the spleen volume

Now, go back and calculate the volume of the segmented spleen.

**Hints**:
- Write a new operator to take the segmentation as an in-memory image, then count the number of non-zero pixels.
- You can use the code below as a starting point 

In [ ]:
@md.input #<---- what should be the input ?> 
@md.output # <--- what should be the output 
@md.env(pip_packages=["monai==0.8.0", "torch>=1.5", "numpy>=1.20", "nibabel"])
class CalculateVolumeOperator(Operator):
    def __init__(self):
        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        print("hello from my new operator")

You will also need to add a flow in the `compose` function to connect the `AISpleenSegApp` operator to the `CalculateVolumeOperator`.

You'll find the code below commented out in your current compose function.  You can uncomment the code after you've written your CalculateVolumeOperator function to test your solution.

```
volume_calculate_operator=CalculateVolumeOperator()
self.add_flow(spleen_seg_op,volume_calculate_operator, {"seg_image": "seg_image"})
```



Full working code for the `CalculateVolumeOperator` is given below.

In [ ]:
@md.input("seg_image", Image, IOType.IN_MEMORY)
@md.output("result_text", Text, IOType.IN_MEMORY)
@md.env(pip_packages=["monai==0.8.0", "torch>=1.5", "numpy>=1.20", "nibabel"])
class CalculateVolumeOperator(Operator):
    def __init__(self):
        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        print("hello from my new operator")
        
        seg = op_input.get("seg_image")
        # In case the Image object is not in the input, and input is the seg image file folder path.
        if not isinstance(seg, Image):
            raise ValueError("Input 'seg_image' is not Image or DataPath.")
        print (f"------got seg")
        seg_np = seg.asnumpy()
        print (f"shape is {seg_np.shape}")
        count=np.count_nonzero(seg_np)
        print (f"------pixel count is {count}")
        op_output.set(str(count), "result_text")

### 7.4 Write the spleen volume to DICOM SR

Now that you have calculated the spleen volume, you can write it to a DICOM SR.

**Hints**:
- Write the count from the `CalculateVolumeOperator` to the DICOM SR text field 
- Add flow from `CalculateVolumeOperator` to `dicomSR` operator
- You can copy the DICOM SR from the previous notebook or uncomment the relevant lines in your `AISpleenSegApp` compose function for Exercise #4

<a id='#conclusion'></a>
## Conclusion

In this notebook, we have walked through creating a segmentation task and utilizing the existing DICOM Operators provided by MONAI Deploy App SDK. You've run the application in Jupyter, locally using Python and the MONAI Deploy CLI, and finally, you packaged the application using docker and executed the newly created container image.

You also worked on some additional exercises to help you get familiar with working in the MONAI Deploy Workflow.

### What's Next
You're now ready to use MONAI Deploy on your project or start contributing to the project.

[MONAI Deploy Repo](https://github.com/Project-MONAI/monai-deploy)